<a href="https://mng.bz/8wdg" target="_blank">
    <img src="../../Assets/Images/NewMEAPHeader.png" alt="New MEAP" style="width: 100%;" />
</a>


# Chapter 04 - Generation Pipeline: Generating Contextual LLM Responses

### Welcome to chapter 3 of A Simple Introduction to Retrieval Augmented Generation.

In this chapter, we introduce the concepts behind the real-time generation pipeline that uses the knowledge base created by the indexing pipeline. This will complete the development of a simple RAG system.

The generation pipeline consists of three steps -

1. Retrieval
2. Augmentation
3. Generation

<img src="../../Assets/Images/4.1.png" width=800>

## Installing Dependencies

Installing all dependencies. While in the book, you'll find that libraries have been imported when their need arises, here we install all dependencies at one place for the sake of a clean code.

In [66]:
%pip install --upgrade pip --quiet
%pip install langchain==0.2.11 --quiet
%pip install bs4==0.0.2 --quiet
%pip install html2text==2024.2.26 --quiet
%pip install lxml==5.2.2 --quiet
%pip install sentence-transformers==3.0.1 --quiet
%pip install langchain_openai==0.1.17 --quiet
%pip install faiss-cpu==1.8.0.post1 --quiet
%pip install langchain-community==0.2.10 --quiet
%pip install tqdm==4.66.4 --quiet
%pip install langchain-huggingface==0.0.3 --quiet
%pip install scikit-learn==1.4.2 --quiet
%pip install rank_bm25==0.2.2 --quiet


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## 1. Load the Vector Index

In Chapter 3, we were working on indexing the Wikipedia page for the 2023 cricket world cup. If you recall we had used embeddings from OpenAI to encode the text and used FAISS as the vector index to store the embeddings. We also stored the FAISS index in a local directory. Let’s reuse this index

In [67]:
# Import FAISS class from vectorstore library
from langchain_community.vectorstores import FAISS

# Import OpenAIEmbeddings from the library
from langchain_openai import OpenAIEmbeddings

# Set the OPENAI_API_KEY as the environment variable
import os
os.environ["OPENAI_API_KEY"] = "YOUR API KEY" #Imp : Replace with an OpenAI API Key

# Instantiate the embeddings object
embeddings=OpenAIEmbeddings(model="text-embedding-3-large")

# Load the database stored in the local directory
db=FAISS.load_local("../../Assets/Data", embeddings, allow_dangerous_deserialization=True)


## 2. Retrieval

We will now retrieve a relevant passage from the knowledge base that is pertinent to our query - __"Who won the 2023 Cricket World Cup?"__

<img src="../../Assets/Images/4.2.png" width=800>

In [68]:
# Original Question
query = "Who won the 2023 Cricket World Cup?"
# Ranking the chunks in descending order of similarity
docs = db.similarity_search(query)
# Printing the top ranked chunk
print(docs[0].page_content)

2023 ICC Men's Cricket World CupDates5 October – 19 November 2023Administrator(s)International Cricket CouncilCricket formatOne Day International (ODI)Tournament format(s)Round-robin and knockoutHost(s) IndiaChampions Australia (6th title)Runners-up IndiaParticipants10Matches48Attendance1,250,307 (26,048 per match)Player of the series Virat KohliMost runs Virat Kohli (765)Most wickets Mohammed Shami (24)Official websitecricketworldcup.com Highlighted are the countries that participated in the 2023 Cricket World Cup. Means of qualification Date Venue Berths Qualified Host nation — — 1  India ICC Super League 30 July 2020 – 14 May 2023 Various 7 Qualifier 18 June 2023 – 9 July 2023 Zimbabwe 2 Total 10 Location Stadium Capacity[19] No. of matches Ahmedabad Narendra Modi Stadium 132,000 5 Bangalore M. Chinnaswamy Stadium 33,800 5 Chennai M. A. Chidambaram Stadium 38,200 5 Delhi Arun Jaitley Stadium 35,200 5 Dharamshala HPCA Stadium 21,200 5 Hyderabad Rajiv Gandhi International Cricket


This is the most basic implementation of a retriever in the generation pipeline of a RAG-enabled system. This method of retrieval is enabled by embeddings. We used the text-embedding-3-large from OpenAI. FAISS calculated the similarity score based on these embeddings

## 3. Augmentation

The information fetched by the retriever should also be sent to the LLM in form of a natural language prompt. This process of combining the user query and the retrieved information is called augmentation.

<img src="../../Assets/Images/4.3.png" width=800>

We will now execute augmentation with a simple contextual prompt with controlled generation.

In [69]:
# Selecting first chunk as the retrieved information
retrieved_context=docs[0]

# Creating the prompt
augmented_prompt=f"""

Given the context below answer the question.

Question: {query} 

Context : {retrieved_context}

Remember to answer only based on the context provided and not from any other source. 

If the question cannot be answered based on the provided context, say I don’t know.

"""

print(augmented_prompt)



Given the context below answer the question.

Question: Who won the 2023 Cricket World Cup? 

Context : page_content='2023 ICC Men's Cricket World CupDates5 October – 19 November 2023Administrator(s)International Cricket CouncilCricket formatOne Day International (ODI)Tournament format(s)Round-robin and knockoutHost(s) IndiaChampions Australia (6th title)Runners-up IndiaParticipants10Matches48Attendance1,250,307 (26,048 per match)Player of the series Virat KohliMost runs Virat Kohli (765)Most wickets Mohammed Shami (24)Official websitecricketworldcup.com Highlighted are the countries that participated in the 2023 Cricket World Cup. Means of qualification Date Venue Berths Qualified Host nation — — 1  India ICC Super League 30 July 2020 – 14 May 2023 Various 7 Qualifier 18 June 2023 – 9 July 2023 Zimbabwe 2 Total 10 Location Stadium Capacity[19] No. of matches Ahmedabad Narendra Modi Stadium 132,000 5 Bangalore M. Chinnaswamy Stadium 33,800 5 Chennai M. A. Chidambaram Stadium 38,200 5

## 4. Generation

Generation is the final step of this pipeline. While LLMs may be used in any of the previous steps in the pipeline, the generation step is completely reliant on the LLM. The most popular LLMs are the ones being developed by OpenAI, Anthropic, Meta, Google, Microsoft and Mistral amongst other developers. 

We have built a simple retriever using FAISS and OpenAI embeddings and, we created a simple augmented prompt. Now we will use OpenAI’s latest model, GPT-4o, to generate the response.

In [70]:
# Importing the OpenAI library
from openai import OpenAI

# Instantiate the OpenAI client
client = OpenAI()

# Make the API call passing the augmented prompt to the LLM
response = client.chat.completions.create(
  model="gpt-4o",
  messages=	[
    {"role": "user", "content": augmented_prompt}
  		]
)

# Extract the answer from the response object
answer=response.choices[0].message.content

print(answer)


Australia won the 2023 Cricket World Cup.


## 5. A single chunk of code

We can just put all of the above in a single sequence of code

In [71]:
# location of the stored vector index created by the indexing pipeline
db_path='../../Assets/Data'

#Original Question
query="Who won the 2023 cricket world cup?"

In [72]:
# Instantiate the embeddings object
embeddings=OpenAIEmbeddings(model="text-embedding-3-large")

# Load the database stored in the local directory
db=FAISS.load_local(db_path, embeddings, allow_dangerous_deserialization=True)

# Ranking the chunks in descending order of similarity
docs = db.similarity_search(query)

# Selecting first chunk as the retrieved information
retrieved_context=docs[0]

# Creating the prompt
augmented_prompt=f"""

Given the context below answer the question.

Question: {query} 

Context : {retrieved_context}

Remember to answer only based on the context provided and not from any other source. 

If the question cannot be answered based on the provided context, say I don’t know.

"""
# Importing the OpenAI library
from openai import OpenAI

# Instantiate the OpenAI client
client = OpenAI()

# Make the API call passing the augmented prompt to the LLM
response = client.chat.completions.create(
  model="gpt-4o",
  messages=	[
    {"role": "user", "content": augmented_prompt}
  		]
)

# Extract the answer from the response object
answer=response.choices[0].message.content

print(answer)


Australia won the 2023 Cricket World Cup.


And there it is. We have built a generation pipeline, albeit a very simple one. It can now fetch information from the knowledge base and generate an answer that is pertinent to the question asked and rooted in the knowledge base. 

But, hang on! Let's try asking more questions.

# 6. RAG function

Let us build a function that will take two inputs - 
1. User Query
2. Location of the Vector Index (Knowledge base)

And generate an answer

#### Retrieval Function

In [73]:
def retrieve_context(query, db_path):
    embeddings=OpenAIEmbeddings(model="text-embedding-3-large")

    # Load the database stored in the local directory
    db=FAISS.load_local(db_path, embeddings, allow_dangerous_deserialization=True)

    # Ranking the chunks in descending order of similarity
    docs = db.similarity_search(query)
    # Selecting first chunk as the retrieved information
    retrieved_context=docs[0].page_content

    return str(retrieved_context)

In [74]:
print(retrieve_context(query, db_path))

2023 ICC Men's Cricket World CupDates5 October – 19 November 2023Administrator(s)International Cricket CouncilCricket formatOne Day International (ODI)Tournament format(s)Round-robin and knockoutHost(s) IndiaChampions Australia (6th title)Runners-up IndiaParticipants10Matches48Attendance1,250,307 (26,048 per match)Player of the series Virat KohliMost runs Virat Kohli (765)Most wickets Mohammed Shami (24)Official websitecricketworldcup.com Highlighted are the countries that participated in the 2023 Cricket World Cup. Means of qualification Date Venue Berths Qualified Host nation — — 1  India ICC Super League 30 July 2020 – 14 May 2023 Various 7 Qualifier 18 June 2023 – 9 July 2023 Zimbabwe 2 Total 10 Location Stadium Capacity[19] No. of matches Ahmedabad Narendra Modi Stadium 132,000 5 Bangalore M. Chinnaswamy Stadium 33,800 5 Chennai M. A. Chidambaram Stadium 38,200 5 Delhi Arun Jaitley Stadium 35,200 5 Dharamshala HPCA Stadium 21,200 5 Hyderabad Rajiv Gandhi International Cricket


#### Augmentation Function

In [75]:
def create_augmeted(query, db_path):

    retrieve_context(query,db_path)

    # Creating the prompt
    augmented_prompt=f"""

    Given the context below answer the question.

    Question: {query} 

    Context : {retrieved_context}

    Remember to answer only based on the context provided and not from any other source. 

    If the question cannot be answered based on the provided context, say I don’t know.

    """

    return str(augmented_prompt)

In [76]:
print(create_augmeted(query, db_path))



    Given the context below answer the question.

    Question: Who won the 2023 cricket world cup? 

    Context : page_content='2023 ICC Men's Cricket World CupDates5 October – 19 November 2023Administrator(s)International Cricket CouncilCricket formatOne Day International (ODI)Tournament format(s)Round-robin and knockoutHost(s) IndiaChampions Australia (6th title)Runners-up IndiaParticipants10Matches48Attendance1,250,307 (26,048 per match)Player of the series Virat KohliMost runs Virat Kohli (765)Most wickets Mohammed Shami (24)Official websitecricketworldcup.com Highlighted are the countries that participated in the 2023 Cricket World Cup. Means of qualification Date Venue Berths Qualified Host nation — — 1  India ICC Super League 30 July 2020 – 14 May 2023 Various 7 Qualifier 18 June 2023 – 9 July 2023 Zimbabwe 2 Total 10 Location Stadium Capacity[19] No. of matches Ahmedabad Narendra Modi Stadium 132,000 5 Bangalore M. Chinnaswamy Stadium 33,800 5 Chennai M. A. Chidambaram Stad

#### RAG function

In [77]:
def create_rag(query, db_path):

    augmented_prompt=create_augmeted(query,db_path)

    # Importing the OpenAI library
    from openai import OpenAI

    # Instantiate the OpenAI client
    client = OpenAI()

    # Make the API call passing the augmented prompt to the LLM
    response = client.chat.completions.create(
    model="gpt-4o",
    messages=	[
        {"role": "user", "content": augmented_prompt}
  		]
    )

    # Extract the answer from the response object
    answer=response.choices[0].message.content

    return answer


Let's try sending our question to this function.

In [78]:
create_rag("Who won the 2023 Cricket World Cup?","../../Assets/Data" )

'Australia won the 2023 Cricket World Cup.'

Let's ask another one.

In [79]:
create_rag("What was Virat Kohli's achievement in the Cup?","../../Assets/Data" )

"Virat Kohli's achievement in the Cup was being the Player of the Series and scoring the most runs (765) in the tournament."

We can also ask a list of questions and see what the responses are

In [80]:
list_of_queries=['What was the outcome of the match between Australia and the Netherlands on 25 October 2023?',
 'What ongoing cricket competition is currently taking place that involves multiple international teams?',
 'What was the deadline for teams to finalize their 15-player squads for the 2023 Cricket World Cup?',
 "What were the key highlights of the 2023 ICC Men's Cricket World Cup?",
 'What were the key outcomes of the 2023 Cricket World Cup, including the final match results and notable player statistics?',
 'What years had Cricket World Cup finals and their host nations?',
 "Which org has managed the Cricket World Cup since '75?",
 "What was India's winning margin vs. S. Africa on Nov 5, 2023?",
 'What teams qualified for the semi-finals in the 2023 Cricket World Cup?']

In [81]:
for query in list_of_queries:
    print(f"Query:{query}")
    print(f"Response: {create_rag(query,db_path)} \n")


Query:What was the outcome of the match between Australia and the Netherlands on 25 October 2023?
Response: I don’t know. 

Query:What ongoing cricket competition is currently taking place that involves multiple international teams?
Response: The ongoing cricket competition that is currently taking place and involves multiple international teams is the 2023 ICC Men's Cricket World Cup. 

Query:What was the deadline for teams to finalize their 15-player squads for the 2023 Cricket World Cup?
Response: I don’t know. 

Query:What were the key highlights of the 2023 ICC Men's Cricket World Cup?
Response: The key highlights of the 2023 ICC Men's Cricket World Cup include:

- Dates: The tournament took place from 5 October to 19 November 2023.
- Host: The tournament was hosted by India.
- Champions: Australia won the tournament, claiming their 6th title.
- Runners-up: India finished as the runners-up.
- Number of participants: There were 10 participating teams.
- Number of matches: A total o

For some of the questions above, the response may be "I don't know". That is when the LLM can't find an answer in the retrieved context. In our augmentation step, we had asked the LLM to do so.

---

Is the RAG system that we have created generating the responses on the expected lines? Is the LLM still hallucinating? Before trying to improve the performance of the system we need to be able to measure and benchmark it. That is what we will do in chapter 5. We will look at the evaluation metrics and the popular benchmarks for RAG.

---

<img src="../../Assets/Images/profile_s.png" width=100> 

Hi! I'm Abhinav! I am an entrepreneur and Vice President of Artificial Intelligence at Yarnit. I have spent over 15 years consulting and leadership roles in data science, machine learning and AI. My current focus is in the applied Generative AI domain focussing on solving enterprise needs through contextual intelligence. I'm passionate about AI advancements constantly exploring emerging technologies to push the boundaries and create positive impacts in the world. Let’s build the future, together!

[If you haven't already, please subscribe to the MEAP of A Simple Guide to Retrieval Augmented Generation here](https://mng.bz/8wdg)

<a href="https://mng.bz/8wdg" target="_blank">
    <img src="../../Assets/Images/NewMEAPFooter.png" alt="New MEAP" style="width: 100%;" />
</a>

#### If you'd like to chat, I'd be very happy to connect

[![GitHub followers](https://img.shields.io/badge/Github-000000?style=for-the-badge&logo=github&logoColor=black&color=orange)](https://github.com/abhinav-kimothi)
[![LinkedIn](https://img.shields.io/badge/LinkedIn-000000?style=for-the-badge&logo=linkedin&logoColor=orange&color=black)](https://www.linkedin.com/comm/mynetwork/discovery-see-all?usecase=PEOPLE_FOLLOWS&followMember=abhinav-kimothi)
[![Medium](https://img.shields.io/badge/Medium-000000?style=for-the-badge&logo=medium&logoColor=black&color=orange)](https://medium.com/@abhinavkimothi)
[![Insta](https://img.shields.io/badge/Instagram-000000?style=for-the-badge&logo=instagram&logoColor=orange&color=black)](https://www.instagram.com/akaiworks/)
[![Mail](https://img.shields.io/badge/email-000000?style=for-the-badge&logo=gmail&logoColor=black&color=orange)](mailto:abhinav.kimothi.ds@gmail.com)
[![X](https://img.shields.io/badge/Follow-000000?style=for-the-badge&logo=X&logoColor=orange&color=black)](https://twitter.com/abhinav_kimothi)
[![Linktree](https://img.shields.io/badge/Linktree-000000?style=for-the-badge&logo=linktree&logoColor=black&color=orange)](https://linktr.ee/abhinavkimothi)
[![Gumroad](https://img.shields.io/badge/Gumroad-000000?style=for-the-badge&logo=gumroad&logoColor=orange&color=black)](https://abhinavkimothi.gumroad.com/)

---